In [3]:
from androguard.misc import AnalyzeAPK
a, d, dx = AnalyzeAPK("../data/droidkungfu_w_benign/5/benign/87057929EFD04326DA2A553FF91B9027AF1AFDE0A6168046F18966F950CA0AF5.apk")

In [4]:
print(a)

In [5]:
for meth in dx.get_permission_usage('android.permission.SEND_SMS', a.get_effective_target_sdk_version()):
    print("Using API method {}".format(meth))
    print("used in:")
    for _, m, _ in meth.get_xref_from():
        print(m.full_name)

AttributeError: 'Analysis' object has no attribute 'get_permission_usage'

In [7]:
CG = dx.get_call_graph(classname, methodname, descriptor, accessflag, no_isolated, entry_points)

NameError: name 'classname' is not defined